对于许多机器学习算法，首先对数据进行缩放，然后手动合并特征，再利用无监督机器学习来学习特征。因此，大多数机器学习应用不仅需要应用单个算法，而且还需要将许多不同的处理步骤和机器学习模型链接在一起。本章将介绍如何使用 Pipeline 类来简化构建变换和模型链的过程。我们将重点介绍如何将 Pipeline 和 GridSearchCV 结合起来，从而同时搜索所有处理步骤中的参数。

举一个例子来说明模型链的重要性。我们知道，可以通过使用 MinMaxScaler 进行预处理来大大提高核SVM在 cancer 数据集上的性能。下面这些代码实现了划分数据、计算最小值和最大值、缩放数据与训练 SVM：

In [9]:
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)

scaler = MinMaxScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC()

svm.fit(X_train_scaled, y_train)
svm.score(X_test_scaled, y_test)

0.972027972027972

## 用预处理进行参数选择

In [16]:
# 只是为了便于说明，不要在实践中使用这些代码！
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'gamma': [0.001, 0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(svm, param_grid=param_grid, cv=5)
# 利用缩放后的数据对SVC参数进行网格搜索
grid.fit(X_train_scaled, y_train)

print("交叉验证分数:", grid.best_score_)
print("测试集分数:", grid.score(X_test_scaled, y_test))
print("最优参数:", grid.best_params_)

交叉验证分数: 0.9812311901504789
测试集分数: 0.972027972027972
最优参数: {'C': 1, 'gamma': 1}


这里我们利用缩放后的数据对SVC参数进行网格搜索。但是，这样有一个不易察觉的陷阱。在缩放数据时，我们使用了训练集中的所有数据，然后，我们使用缩放后的训练数据来运行带交叉验证的网格搜索。但是交叉验证会将训练集划分为训练部分和验证部分(测试部分），我们在缩放数据的时候已经使用过验证部分所包含的信息，因此，我们已经将验证部分的数据信息泄露（leak）给建模过程。这将导致在交叉验证过程中得到过于乐观的结果，并可能导致选择次优的参数。

<img src="https://gitee.com/itangqian/picgo/raw/master/img/image-20210412103425022.png" alt="image-20210412103425022" style="zoom: 33%;" />

为了解决这个问题，在交叉验证的过程中，应该在进行任何预处理之前完成数据集的划分。任何从数据集中提取信息的处理过程都应该仅应用于数据集的训练部分，因此，任何交叉验证都应该位于处理过程的“最外层循环”。

在scikit-learn中，要想使用`cross_val_score` 函数和`GridSearchCV`函数实现这一点，可以使用Pipeline类。Pipeline 类可以将多个处理步骤合并(glue) 为单个scikit-learn估计器。Pipeline 类本身具有fit、predict 和score方法，其行为与scikit-learn中的其他模型相同。Pipeline 类最常见的用例是将预处理步骤(比如数据缩放)与一个监督模型(比如分类器)链接在一起。

## 构建管道

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)
# 创建两个步骤，数据处理MinMaxScaler，分类器SVC
pipe = Pipeline([('scaler', MinMaxScaler()), ("svm", SVC())])
# pipe.fit首先对第一个步骤（数据处理/缩放器）调用fit，然后使用该缩放器对训练数据进行变换，最后用缩放后的数据来拟合SVM。
pipe.fit(X_train, y_train)
# 在测试集上进行评估，首先使用缩放器对测试数据进行变换，然后利用缩放后的测试数据对SVM调用score方法。
# 这个结果和开头手动进行数据变换的结果是一样的
# 利用管道，我们减少了“预处理+分类”过程所需要的代码量
# 但是，使用管道的主要优点是，我们可以在`cross_val_score`或`GridSearchCV`使用这个估计器。
pipe.score(X_test, y_test)

0.972027972027972

## 在网格搜索中使用管道

在网格搜索中使用管道的工作原理与使用任何其他估计器都相同。我们定义一个需要搜索的参数网格，并利用管道和参数网格构建一个GridSearchCV。不过在指定参数网格时存在一处细微的变化。我们需要为每个参数指定它在管道中所属的步骤。我们要调节的两个参数C和gamma都是SVC的参数，属于第二个步骤。我们给这个步骤的名称是"svm"。 为管道定义参数网格的语法是为每个参数指定步骤名称，后面加上`__`(双下划线)，然后是参数名称。因此，要想搜索SVC的C参数，必须使用`svm__C`作为参数网格字典的键，对gamma参数也是同理:

In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)
# 创建两个步骤，数据处理MinMaxScaler，分类器SVC
pipe = Pipeline([('scaler', MinMaxScaler()), ("svm", SVC())])

param_grid = {'svm__C': [0.001, 0.01, 0.1, 1, 10, 100],
              'svm__gamma': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print("交叉验证分数:", grid.best_score_)
print("测试集分数:", grid.score(X_test, y_test))
print("最优参数:", grid.best_params_)

交叉验证分数: 0.9812311901504789
测试集分数: 0.972027972027972
最优参数: {'svm__C': 1, 'svm__gamma': 1}


与前面所做的网格搜索不同，现在对于交叉验证的每次划分来说，仅使用训练部分对MinMaxScaler进行拟合，测试部分的信息没有泄露到参数搜索中。

在交叉验证中，信息泄露的影响大小取决于预处理步骤的性质。使用测试部分来估计数据的范围，通常不会产生可怕的影响，但在特征提取和特征选择中使用测试部分，则会导致结果的显著差异。

<img src="https://gitee.com/itangqian/picgo/raw/master/img/image-20210412110113349.png" alt="image-20210412110113349" style="zoom: 33%;" />

## 举例说明信息泄露

我们考虑一个假想的回归任务，包含从高斯分布中独立采样的100个样本与10000个特征。我们还从高斯分布中对响应进行采样:

In [5]:
import numpy as np 
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge


rnd = np.random.RandomState(seed=0)
X = rnd.normal(size=(100, 10000))
y = rnd.normal(size=(100,))

# 考虑到我们创建数据集的方式，数据X与目标y之间没有任何关系(它们是独立的),所以应该不可能从这个数据集中学到任何内容。
# 现在我们将完成下列工作。首先利用SelectPercentile特征选择从10 000个特征中选择信息量最大的特征，然后使用交叉验证对Ridge回归进行评估:
select = SelectPercentile(score_func=f_regression, percentile=5).fit(X, y)
X_selected = select.transform(X)
print(X_selected.shape)
cross_val_score(Ridge(), X_selected, y, cv=5).mean()

(100, 500)


0.9057953065239822

交叉验证计算得到的平均R2为0.91，表示这是一个非常好的模型。这显然是不对的,因为我们的数据是完全随机的。这里的特征选择从10 000个随机特征中(碰巧)选出
了与目标相关性非常好的一些特征。由于我们在交叉验证**之外**对特征选择进行拟合，所以它能够找到在训练部分和测试部分都相关的特征。从测试部分泄露出去的信息包
含的信息量非常大，导致得到非常不切实际的结果。我们将这个结果与正确的交叉验证(使用管道)进行对比:

In [4]:
import numpy as np 
from sklearn.feature_selection import SelectPercentile, f_regression
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline


rnd = np.random.RandomState(seed=0)
X = rnd.normal(size=(100, 10000))
y = rnd.normal(size=(100,))

pipe = Pipeline([('select', SelectPercentile(score_func=f_regression, percentile=5)), ('ridge', Ridge())])
cross_val_score(pipe, X, y, cv=5).mean()

-0.24655422384952802

这一次我们得到了负的R2分数，表示模型很差。利用管道，特征选择现在位于交叉验证循环内部。也就是说，仅使用数据的训练部分来选择特征，而不使用测试部分。特征选择找到的特征在训练集中与目标相关，但由于数据是完全随机的，这些特征在测试集中并不与目标相关。在这个例子中，修正特征选择中的数据泄露问题，结论也由“模型表现很好”变为“模型根本没有效果”。

## 用make_pipeline方便地创建管道

利用上述语法创建管道有时有点麻烦，我们通常不需要为每一个步骤提供用户指定的名称。有一个很方便的函数`make_pipeline`， 可以为我们创建管道并根据每个步骤所属的类为其自动命名。`make_pipeline` 的语法如下所示:

In [10]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

pipe_long = Pipeline([('scaler', MinMaxScaler()), ('svc', SVC(C=100))])
pipe_short = make_pipeline(MinMaxScaler(), SVC(C=100))  
# 通过steps属性查看步骤的名称
pipe_short.steps

[('minmaxscaler', MinMaxScaler()), ('svc', SVC(C=100))]

这两个步骤被命名为minmaxscaler和svc。一般来说，步骤名称只是类名称的小写版本。如果多个步骤属于同一个类，则会附加一个数字。

## 访问网格搜索管道中的属性

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

cancer = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)

pipe = make_pipeline(StandardScaler(), LogisticRegression(max_iter=300))
# 由于我们使用了make_ pipeline 函数，所以管道中LogisticRegression步骤的名称是小写的类名称logisticregression。
# 因此，为了调节参数C，我们必须指定logisticregression__C的参数网格:
param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print(grid.best_estimator_)
# 我们可以使用管道的named_steps属性来访问logisticregression步骤
print(grid.best_estimator_.named_steps["logisticregression"])
# 现在我们得到了训练过的LogisticRegression实例，下面我们可以访问与每个输入特征相关的系数(权重) 
print(grid.best_estimator_.named_steps["logisticregression"].coef_)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression(C=1, max_iter=300))])
LogisticRegression(C=1, max_iter=300)
[[-0.29792942 -0.58056355 -0.3109406  -0.377129   -0.11984232  0.42855478
  -0.71131106 -0.85371164 -0.46688191  0.11762548 -1.38262136  0.0899184
  -0.94778563 -0.94686238  0.18575731  0.99305313  0.11090349 -0.3458275
   0.20290919  0.80470317 -0.91626377 -0.91726667 -0.8159834  -0.86539197
  -0.45539191  0.10347391 -0.83009341 -0.98445173 -0.5920036  -0.61086989]]


## 网格搜索预处理步骤与模型参数

我们可以利用管道将机器学习工作流程中的所有处理步骤封装成一个scikit-learn估计器。这么做的另一个好处在于，现在我们可以使用监督任务(比如回归或分类)的输出来调节预处理参数。下面我们用一个管道来包含3个步骤:缩放数据、计算多项式特征与岭回归, 看看我们选择几次多项式？

In [37]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, random_state=0)
pipe = make_pipeline(StandardScaler(), PolynomialFeatures(), Ridge())

param_grid = {'polynomialfeatures__degree': [1, 2, 3],
              'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
# 由此可见，选择2次多项式
print(grid.best_params_)
print(grid.score(X_test, y_test))

{'polynomialfeatures__degree': 2, 'ridge__alpha': 10}
0.7683045464100143


为了对比，我们运行一个没有多项式特征的网格搜索

In [38]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge

boston = load_boston()
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, random_state=0)
pipe = make_pipeline(StandardScaler(), Ridge())

param_grid = {'ridge__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
# 由此可见，选择2次多项式
print(grid.best_params_)
print(grid.score(X_test, y_test))

{'ridge__alpha': 10}
0.6272671927504456


正与我们所预料的那样，不使用多项式特征得到了明显更差的结果。

同时搜索预处理参数与模型参数是一个非常强大的策略。但是要记住，GridSearchCV 会尝试指定参数的所有可能组合。因此，向网格中添加更多参数，需要构建的模型数量将呈指数增长。

## 网格搜索选择使用哪个模型

下面是一个例子:在iris数据集上比较RandomForestClassifier和SVC。 我们知道，SVC可能需要对数据进行缩放，所以我们还需要搜索是使用StandardScaler还是不使用预处理。我们知道，RandomForestClassifier 不需要预处理。我们先定义管道。这里我们显式地对步骤命名。我们需要两个步骤，一个用于预处理，然后是一个分类器。我们可以用SVC和StandardScaler来将其实例化。

现在我们可以定义需要搜索的parameter_grid。 我们希望classifier是RandomForestClassifier或SVC。由于这两种分类器需要调节不同的参数,并且需要不同的预处理，为了将-个估计器分配给一个步骤，我们使用步骤名称作为参数名称。如果我们想跳过管道中的某个步骤(例如，RandomForest 不需要预处理)，则可以将该步骤设置为None。

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()

pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', SVC())])
param_grid = [
         {'classifier': [SVC()], 'preprocessing': [StandardScaler(), None],
          'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
          'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
         {'classifier': [RandomForestClassifier(n_estimators=100)],
          'preprocessing': [None], 'classifier__max_features': [1, 2, 3]}]
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, random_state=0)
grid = GridSearchCV(pipe, param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best params:{}".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

Best params:{'classifier': SVC(C=10, gamma=0.01), 'classifier__C': 10, 'classifier__gamma': 0.01, 'preprocessing': StandardScaler()}
Best cross-validation score: 0.99
Test-set score: 0.98
